In [1]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt

In [2]:
def rotation(theta):
	return np.array([
		[np.cos(theta), -np.sin(theta), 0],
		[np.sin(theta), np.cos(theta), 0],
		[0, 0, 1]
	])

def translation(pos):
	return np.array([
		[1, 0, pos[0]],
		[0, 1, pos[1]],
		[0, 0, 1]
	])

def scaling(s):
	return np.array([
		[s[0], 0, 0],
		[0, s[1], 0],
		[0, 0, 1]
	])

# 1. Translación, rotación, y escalado

In [6]:
class AffineImage:
	def __init__(self, image, title):
		self.image = image
		self.title = title
		cv.namedWindow(title)
		self.pos = np.array([0, 0])
		self.rot = 0
		self.scale = np.array([1, 1], dtype=np.float32)
		self.focus = np.array(image.shape[:2]) / 2

	def transformed(self):
		rot = self.rot
		pos = self.pos
		s = self.scale
		h, w = self.image.shape[:2]
		norm = translation(-self.focus)
		denorm = translation(self.focus)
		affine = (translation(pos) @ denorm @ rotation(rot) @ scaling(s) @ norm)
		return cv.warpAffine(self.image, affine[:2], (w, h))

	def show(self):
		cv.imshow(self.title, self.transformed())



In [7]:
from math import pi
image = AffineImage(cv.imread('tears.png'), 'tears')

def pos_x(val):
	image.pos[0] = val - 50
	image.show()

def pos_y(val):
	image.pos[1] = val - 50
	image.show()

def rotate(degrees):
	radians = degrees * pi / 180
	image.rot = radians
	image.show()

def scale_x(deciscale):
	image.scale[0] = float(deciscale / 10.)
	image.show()

def scale_y(deciscale):
	image.scale[0] = float(deciscale / 10.)
	image.show()

def focus_cb(event, x, y, flag, param):
	if event == cv.EVENT_LBUTTONDBLCLK:
		image.focus = np.array([x, y])
		image.show()

cv.createTrackbar("x", image.title, 50, 100, pos_x)
cv.createTrackbar("y", image.title, 50, 100, pos_y)
cv.createTrackbar("degrees", image.title, 0, 360, rotate)
cv.createTrackbar("scale x", image.title, 10, 100, scale_x)
cv.createTrackbar("scale y", image.title, 10, 100, scale_y)
cv.setMouseCallback(image.title, focus_cb)

image.show()

cv.waitKey(0)
cv.destroyAllWindows()


QObject::moveToThread: Current thread (0xd0b4ea0) is not the object's thread (0xcf066b0).
Cannot move to target thread (0xd0b4ea0)

QObject::moveToThread: Current thread (0xd0b4ea0) is not the object's thread (0xcf066b0).
Cannot move to target thread (0xd0b4ea0)

QObject::moveToThread: Current thread (0xd0b4ea0) is not the object's thread (0xcf066b0).
Cannot move to target thread (0xd0b4ea0)

QObject::moveToThread: Current thread (0xd0b4ea0) is not the object's thread (0xcf066b0).
Cannot move to target thread (0xd0b4ea0)

QObject::moveToThread: Current thread (0xd0b4ea0) is not the object's thread (0xcf066b0).
Cannot move to target thread (0xd0b4ea0)

QObject::moveToThread: Current thread (0xd0b4ea0) is not the object's thread (0xcf066b0).
Cannot move to target thread (0xd0b4ea0)

QObject::moveToThread: Current thread (0xd0b4ea0) is not the object's thread (0xcf066b0).
Cannot move to target thread (0xd0b4ea0)

QObject::moveToThread: Current thread (0xd0b4ea0) is not the object's thread

# 2. Perspectiva

In [33]:
class PerspectiveImage:
	def __init__(self, image, title) -> None:
		self.image = image
		self.title = title
		self.perspective_points = []
		self.mouse_pos = np.zeros((2,))

	def rendered(self):
		match self.perspective_points:
			case pts if len(pts) == 0:
				return self.image
			case pts if len(pts) >= 4:
				h, w = self.image.shape[:2]
				from_points = np.float32([(0, 0), (0, h), (h, w), (w, 0)])
				to_points = np.float32(self.perspective_points)
				tp = cv.getPerspectiveTransform(from_points, to_points)
				return cv.warpPerspective(self.image, tp, (h, w))
			case pts:
				line_img = self.image.copy()
				for pt0, pt1 in zip(pts, pts[1:]):
					line_img = cv.line(line_img, pt0, pt1, (0, 255, 0), 2)
				line_img = cv.line(line_img, pts[-1], self.mouse_pos, (0, 255, 0), 2)
				if len(pts) == 3:
					line_img = cv.line(line_img, self.mouse_pos, pts[0], (0, 255, 0), 2)
				return line_img

	def show(self):
		cv.imshow(self.title, self.rendered())

	def register_cb(self):
		def cb(event, x, y, *_):
			if event == cv.EVENT_MOUSEMOVE:
				self.mouse_pos = np.array([x, y])
			if event == cv.EVENT_LBUTTONDBLCLK:
				match self.perspective_points:
					case pts if len(pts) >= 4:
						self.perspective_points = []
					case pts:
						self.perspective_points.append((x, y))
			self.show()
		cv.namedWindow(self.title)
		cv.setMouseCallback(self.title, cb)

In [39]:
image = PerspectiveImage(cv.imread("tears.png"), "tears")

image.register_cb()
image.show()

cv.waitKey(0)

cv.destroyAllWindows()

QObject::moveToThread: Current thread (0x1778b8e0) is not the object's thread (0x17889ee0).
Cannot move to target thread (0x1778b8e0)

QObject::moveToThread: Current thread (0x1778b8e0) is not the object's thread (0x17889ee0).
Cannot move to target thread (0x1778b8e0)

QObject::moveToThread: Current thread (0x1778b8e0) is not the object's thread (0x17889ee0).
Cannot move to target thread (0x1778b8e0)

QObject::moveToThread: Current thread (0x1778b8e0) is not the object's thread (0x17889ee0).
Cannot move to target thread (0x1778b8e0)

QObject::moveToThread: Current thread (0x1778b8e0) is not the object's thread (0x17889ee0).
Cannot move to target thread (0x1778b8e0)

QObject::moveToThread: Current thread (0x1778b8e0) is not the object's thread (0x17889ee0).
Cannot move to target thread (0x1778b8e0)

QObject::moveToThread: Current thread (0x1778b8e0) is not the object's thread (0x17889ee0).
Cannot move to target thread (0x1778b8e0)

QObject::moveToThread: Current thread (0x1778b8e0) is n

# 3. Distorsión

In [14]:
class DistortedImage:
	def __init__(self, image, title):
		self.image = image
		self.title = title
		self.k1 = 0
		self.k2 = 0
		self.focal = 10

	def transformed(self):
		w, h = self.__image.shape[:2]
		dc = np.zeros((4, 1), np.float64)
		dc[0, 0] = self.k1
		dc[1, 0] = self.k2

		cam = np.eye(3, dtype=np.float32)
		cam[0, 2] = w/2
		cam[1, 2] = h/2
		cam[0, 0] = self.focal
		cam[1, 1] = self.focal

		return cv.undistort(self.__image, cam, dc)

	def show(self):
		cv.imshow(self.__title, self.transformed())

In [15]:
image = DistortedImage(cv.imread("tears.png"), "tears")

def k1_tb(k1):
	image.k1 = (k1 - 50) / 1e4
	image.show()

def k2_tb(k2):
	image.k2 = (k2 - 50) / 1e6
	image.show()

def focal_tb(focal):
	image.focal = focal
	image.show()

cv.namedWindow("tears")

cv.createTrackbar("k1", "tears", 50, 100, k1_tb)
cv.createTrackbar("k2", "tears", 50, 100, k2_tb)
cv.createTrackbar("focal length", "tears", 10, 100, focal_tb)

image.show()

cv.waitKey(0)
cv.destroyAllWindows()

QObject::moveToThread: Current thread (0x1778b8e0) is not the object's thread (0x17889ee0).
Cannot move to target thread (0x1778b8e0)

QObject::moveToThread: Current thread (0x1778b8e0) is not the object's thread (0x17889ee0).
Cannot move to target thread (0x1778b8e0)

QObject::moveToThread: Current thread (0x1778b8e0) is not the object's thread (0x17889ee0).
Cannot move to target thread (0x1778b8e0)

QObject::moveToThread: Current thread (0x1778b8e0) is not the object's thread (0x17889ee0).
Cannot move to target thread (0x1778b8e0)

QObject::moveToThread: Current thread (0x1778b8e0) is not the object's thread (0x17889ee0).
Cannot move to target thread (0x1778b8e0)

QObject::moveToThread: Current thread (0x1778b8e0) is not the object's thread (0x17889ee0).
Cannot move to target thread (0x1778b8e0)

QObject::moveToThread: Current thread (0x1778b8e0) is not the object's thread (0x17889ee0).
Cannot move to target thread (0x1778b8e0)

QObject::moveToThread: Current thread (0x1778b8e0) is n